In [ ]:
!pip install --upgrade torch-scatter
!pip install --upgrade torch-sparse
!pip install --upgrade torch-cluster
!pip install --upgrade torch-spline-conv 
!pip install --upgrade torch-geometric
!pip install rdflib
!pip install torch

In [ ]:
import argparse
import os.path as osp
import datetime
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Entities
from torch_geometric.utils import k_hop_subgraph
from torch_geometric.nn import RGCNConv, FastRGCNConv



# parser = argparse.ArgumentParser()
# parser.add_argument('--dataset', type=str,
#                     choices=['AIFB', 'MUTAG', 'BGS', 'AM'])
# args = parser.parse_args()

# # Trade memory consumption for faster computation.
# if args.dataset in ['AIFB', 'MUTAG']:
#     RGCNConv = FastRGCNConv

# path = osp.join(osp.dirname(osp.realpath(__file__)), '..', 'data', 'Entities')
dataset = Entities('/temp/data', 'BGS')
data = dataset[0]

# BGS and AM graphs are too big to process them in a full-batch fashion.
# Since our model does only make use of a rather small receptive field, we
# filter the graph to only contain the nodes that are at most 2-hop neighbors
# away from any training/test node.
test_10 = []
for i in range(10):
  node_idx = torch.cat([data.train_idx, data.test_idx], dim=0)
  node_idx, edge_index, mapping, edge_mask = k_hop_subgraph(node_idx, 2, data.edge_index, relabel_nodes=True)

  data.num_nodes = node_idx.size(0)
  data.edge_index = edge_index
  data.edge_type = data.edge_type[edge_mask]
  data.train_idx = mapping[:data.train_idx.size(0)]
  data.test_idx = mapping[data.train_idx.size(0):]


  class Net(torch.nn.Module):
    def __init__(self):
      super(Net, self).__init__()
      self.conv1 = RGCNConv(data.num_nodes, 16, dataset.num_relations,
                              num_bases=40)
      self.conv2 = RGCNConv(16, dataset.num_classes, dataset.num_relations,
                              num_bases=40)

    def forward(self, edge_index, edge_type):
      x = F.relu(self.conv1(None, edge_index, edge_type))
      x = self.conv2(x, edge_index, edge_type)
      return F.log_softmax(x, dim=1)


  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # device = torch.device('cpu') if args.dataset == 'AM' else device
  model, data = Net().to(device), data.to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0005)


  def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.edge_index, data.edge_type)
    loss = F.nll_loss(out[data.train_idx], data.train_y)
    loss.backward()
    optimizer.step()
    return loss.item()


  @torch.no_grad()
  def test():
    model.eval()
    pred = model(data.edge_index, data.edge_type).argmax(dim=-1)
    train_acc = pred[data.train_idx].eq(data.train_y).to(torch.float).mean()
    test_acc = pred[data.test_idx].eq(data.test_y).to(torch.float).mean()
    return train_acc.item(), test_acc.item()

  # print(device)
  tests = []
  train_start = datetime.datetime.now()
  for epoch in range(1, 51):
    start = datetime.datetime.now()
    loss = train()
    train_acc, test_acc = test()
    end = datetime.datetime.now()
    time =end - start
    # print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Train_acc: {test_acc:.4f}, Time: {str(time)}')
    tests.append(test_acc)
  train_end = datetime.datetime.now()
  train_time = train_end - train_start
  print('\n Training: ' + str(train_time))
  test_10.append(np.mean(tests))
  # print(test_10)

Extracting /temp/data/bgs.tgz
Processing...


http://data.bgs.ac.uk/id/EarthMaterialClass/RockName/+^PYSD does not look like a valid URI, trying to serialize this will break.
http://data.bgs.ac.uk/id/EarthMaterialClass/RockName/+^PYSD does not look like a valid URI, trying to serialize this will break.
http://data.bgs.ac.uk/id/EarthMaterialClass/RockName/+#^PRS does not look like a valid URI, trying to serialize this will break.
http://data.bgs.ac.uk/id/EarthMaterialClass/RockName/+#^PRS does not look like a valid URI, trying to serialize this will break.
http://data.bgs.ac.uk/id/EarthMaterialClass/RockName/+#^PRS does not look like a valid URI, trying to serialize this will break.
http://data.bgs.ac.uk/id/EarthMaterialClass/RockName/+#^RSR does not look like a valid URI, trying to serialize this will break.
http://data.bgs.ac.uk/id/EarthMaterialClass/RockName/+#^PRS does not look like a valid URI, trying to serialize this will break.
http://data.bgs.ac.uk/id/EarthMaterialClass/RockName/+#^RSR does not look like a valid URI, tryin

Done!

 Training: 0:05:53.610493

 Training: 0:05:53.583611

 Training: 0:05:53.430447

 Training: 0:05:53.390317

 Training: 0:05:53.540702


In [ ]:
print(np.mean(test_10)*100)

In [ ]:
from rdflib import Graph

aifb = Graph()
am = Graph()
bgs = Graph()
mutag = Graph()

aifb.parse("/content/aifb_summarized.nt", format="nt")

am.parse("/content/am_summarized.nt", format="nt")
bgs.parse("/content/bgs_summarized.nt", format="nt")
mutag.parse("/content/mutag_summarized.nt", format="nt")

<Graph identifier=Naa2ac01a8c6d46d7894bee9e74436d17 (<class 'rdflib.graph.Graph'>)>

In [ ]:
ls = []
for s, p, o in am.triples((None, None, None)):
  ls.append(p)

In [ ]:
import numpy as np

In [ ]:
# for un in ls
len(ls)

56

NVIDIA Tesla K80 GPU 11GB


In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = Net().to(device)
# data = dataset[0].to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# model.train()
# for epoch in range(200):
#     optimizer.zero_grad()
#     out = model(data)
#     loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
#     loss.backward()
#     optimizer.step()

In [ ]:
times = []

In [ ]:
times.append('AM: ' + str(time))

In [ ]:
times

['MUTAG: 0:00:09.611469', 'AIFB: 0:00:12.125963', 'BGS: 0:05:33.224056']

Mutag summarized in **3s** from 10.2 MB to 44.1 kB (231=x reduction in size)


AM summarized in **1min57s** from 755 MB to 7.79 kB (96,919x reduction in size)


BGS summarized in **2min5s** from 209 MB to 5.94 kB (35,185x reduction in size)


AIFB summarized in **3s** from 5.95 MB to 26 kB (229x reduction in size)


['MUTAG: 0:00:09.611469', 'AIFB: 0:00:12.125963', 'BGS: 0:05:33.224056']
